# Session 11: Data Processing

In [ ]:
import os

# For parallel processing
import parsl
import parsl
from parsl import python_app
from parsl.config import Config
from parsl.channels import LocalChannel
from parsl.executors import HighThroughputExecutor
from parsl.providers import LocalProvider

# helpers
from grouputils import initialize_rasterizer

## Background

![](https://raw.githubusercontent.com/PermafrostDiscoveryGateway/viz-raster/develop/docs/images/raster_tldr.png)

Today, we will create rasters from the regularly gridded staged data from yesterday (vector data, left side of diagram above). Each pixel of the raster will contain a stasticic calculated based on the underlying vector data for that pixel.

The two statistics we calculate are:

- number of IWP per pixel
- proportion of pixel covered by IWP

The vector data on the left are geopackages, which can be read in as simple geodataframes. The rasters take the geometry columns of all the geodataframes, which are all simple polygons, and represent them as pixels. 

Similar to the `TileStager` from the first step of the group project, in this step we will use a `RasterTiler` class, which we initialize using `initialize_rasterizer`.

In [ ]:
iwp_rasterizer = initialize_rasterizer("/home/shares/example-pdg-data")


Let's have a look at the data we need to rasterize first.

In [ ]:
staged_paths = iwp_rasterizer.tiles.get_filenames_from_dir('staged')
print(len(staged_paths), "files to rasterize.")

We can use the `rasterize_vector` method on our `RasterTiler` class object, and pass it the first file in our list of staged files to rasterize just one file.

In [ ]:
iwp_rasterizer.rasterize_vector(staged_paths[0])

This went pretty quickly, but with thousands of files, it would still be beneficial to parallelize. Estimate the length of time it would take to process the files in series below:

In [ ]:
# estimate duration of rasterization process
print(round(4 * len(staged_paths)/60), "minutes")

## Rasterize in Parallel

Given what you know about the limits of parallelization, discuss in your group whether you think this problem is:

- cpu-bound
- memory-bound
- I/O-bound
- network-bound

How does it compare to the parallelization task from yesterday?

Instead of parallelizing this step over all 1,082 files, we will instead process the data in batches of files. Below is a function we can use to make batches of files.

In [ ]:
# Because rasterization is relatively quick, we want each parsl "task" to process a batch of tiles.
def make_batch(items, batch_size):
    # Create batches of a given size from a list of items.
    return [items[i:i + batch_size] for i in range(0, len(items), batch_size)]

Use the `make_batch` function to make batches of 10 items each of our staged data (`staged_paths`).

In [ ]:
# make batches of 10 files
batch_size = 10
batches = make_batch(staged_paths, batch_size)

Now, set up your Parsl executor again, with `max_workers` set at 15.

In [ ]:
# Set up Parsl executor
activate_env = 'workon scomp2023-03-20'
htex_local = Config(
    executors=[
        HighThroughputExecutor(
            max_workers=15,
            provider=LocalProvider(
                worker_init=activate_env
            )
        )
    ],
)

parsl.clear()
parsl.load(htex_local)

Next, set up your Parsl app to run the `rasterize_vector` method in parallel. Remember that Parsl apps cannot rely on global variables or package imports, so you'll need to make sure to pass the app all of the variables it needs.

In [ ]:
# Make a Parsl app that uses the rasterize_vectors method
@python_app
def rasterize(staged_paths, rasterizer):
    # Rasterize a batch of vector files
    return rasterizer.rasterize_vectors(staged_paths, make_parents=False)

Now, execute the app in parallel over all of the batches of files you created previously. Don't forget to add lines to shut down the executor and clear `parsl`.

In [ ]:
# Rasterize the batches in parallel
app_futures = []
for batch in batches:
    app_future = rasterize(batch, iwp_rasterizer)
    app_futures.append(app_future)

# Don't continue to print message until all tiles have been rasterized
[app_future.result() for app_future in app_futures]

htex_local.executors[0].shutdown()
parsl.clear()

Check to make sure that you have the same number of GeoTIFF files as we do staged vector tiles:

In [ ]:
geotiff_paths = iwp_rasterizer.tiles.get_filenames_from_dir('geotiff')
len(geotiff_paths) == len(staged_paths)

We have a 1:1 ratio because we only processed the highest resolution files: those at zoom-level 15.

## Bonus

Thinking back to the group project yesterday, how would you test whether this problem is CPU or I/O bound? One option would be to test whether adding more CPU decreases your computation time. Set up a test on a subset of the staged files (the first 500 is fine) and time the process at different `max_workers`. To make sure you don't overload the server, do not exceed 32 workers in your tests.

Here are the results for three sets of tests where the `max_workers` was increased all the way to 88. We also varied the number of files in each batch, which effectively alters the degree of parallelism in our code. The result is a figure which looks remarkably like Amdahl's Law!

![](images/parallel-performance.png)